In [2]:
import numpy as np
import sys
import pandas as pd
from scipy import stats


Load tinycov coverage files binned at 50bp without overlap and the files containing chromosome name and length (first and second columns). Compute the coverage per chromosome, the fraction of total coverage it represents, and the deviation from what is expected from chromosome length

In [10]:
###LAPTOP
dir_cov= "/mnt/c/Owncloud/Experiments/HiC/Coverage/S288c/"
dir_seqs = "/mnt/c/Owncloud/Sequences/Genomes/Yeast_genome_S288c_R64-2-1_2014/"
dir_hic = "/mnt/c/OwnCloud/Experiments/HiC/"
outdir_hic = "/mnt/c/OwnCloud/Experiments/HiC/"
dir_capt = "/mnt/c/OwnCloud/Experiments/Capture/"
###DESKTOP
dir_cov= "/mnt/d/Science/Owncloud/Experiments/HiC/Coverage/S288c/"
dir_seqs = "/mnt/d/Science/Owncloud/Sequences/Genomes/Yeast_genome_S288c_R64-2-1_2014/"
dir_hic = "/mnt/e/Science/HiC/Contact_files/"
outdir_hic = "/mnt/d/Science/OwnCloud/Experiments/HiC/"
dir_capt = "/mnt/d/Science/OwnCloud/Experiments/Capture/"
#dir = "/mnt/d/Science/Owncloud/Experiments/Capture/"#dir_cov= "/mnt/d/Science/Owncloud/Experiments/HiC/Coverage/S288c/"
#dir_seqs = "/mnt/d/Science/Owncloud/Sequences/Genomes/Yeast_genome_S288c_R64-2-1_2014/"
chrs = pd.read_csv(str(dir_seqs + "S288c_chr_2m_mito_centro_coordinates_cumulated.tsv"), sep = "\t")
chrs = np.array(chrs)
len(chrs)
#print(chrs)
#hic_capture_correspondance = pd.read_csv(str(dir_capt + "HiC_Capture_correspondance.tsv"), sep = "\t")
hic_capture_correspondance = pd.read_csv(str(dir_capt + "test.tsv"), sep = "\t")
capture_sites = pd.read_csv(str(dir_capt + "DpnII_sites_coordinates_URA3_S288c_DSB_LY_for_capture_scripts_short.tsv"), sep = "\t")

head = ("readID", "chr1", "pos1", "chr2", "pos2", "strand1", "strand2", "frag1", "frag2")
#Start the loop that will go down the list of HiC and capture library
for i in np.arange(0, len(hic_capture_correspondance),1): 
    hic_samp = str(hic_capture_correspondance.iloc[i,0])
    hic_pairs = pd.read_csv(str(dir_hic + "Pairs/"+ hic_samp + "_valid_idx_filtered.pairs"), names=head, skiprows=int(len(chrs)+3), sep="\t")
    hic_pairs = np.array(hic_pairs)
    cov_chr=[] #will quantify the total contacts made by each chromosome
    cov_chr_rel=[] #will quantify the fraction of total contacts
    cov_chr_norm=[] #will quantify the fraction of contacts normalized onto the genomic fraction of each chromosome -> gives the chromosome copy number
    contacts = 0 
    total_contacts = len(hic_pairs[:,0])*2
    for chrom in chrs[:,0]:
        contacts = int(sum((len(hic_pairs[(hic_pairs[:,1]==str(chrom)),0]),len(hic_pairs[(hic_pairs[:,3]==str(chrom)),0]))))
        cov_chr.append(contacts) 
        cov_chr_rel.append(float(contacts/total_contacts))
        cov_chr_norm.append(float((contacts/total_contacts)/(chrs[chrs[:,0]==str(chrom),1]/sum(chrs[:,1]))))
    head_hic = np.array(("chr", "length", "Hic_pairs", "Hic_pairs_fraction_total", "Hic_chr_copy_number"))
    cov_hic = np.column_stack((chrs[:,0], chrs[:,1], cov_chr, cov_chr_rel, cov_chr_norm))
    cov_hic = np.row_stack((head_hic, cov_hic))
    print(cov_hic)
    np.savetxt(outdir_hic + "Coverage/per_chr/" + hic_samp + "_coverage_per_chr.tsv", cov_hic, delimiter = "\t", fmt='%s')
    
    
    capt_samp = str(hic_capture_correspondance.iloc[i,1])
    head_capt = np.array(("chr", "length", "Hic_pairs", "Hic_pairs_fraction_total", "Hic_chr_copy_number"))
    for site in np.arange(0, len(capture_sites),1):
        chro = capture_sites.iloc[site,1]
        pos = capture_sites.iloc[site,2]
        capt_pairs = pd.read_csv(str(dir_capt + "Pairs/"+ capt_samp + "_valid_idx_filtered_" + str(chro) + "_" + str(pos) + ".pairs"), names=head, skiprows=int(len(chrs)+3), sep="\t")
        capt_pairs = np.array(capt_pairs)
        cov_chr=[] #will quantify the total contacts made by each chromosome
        cov_chr_rel=[] #will quantify the fraction of total contacts
        cov_chr_norm=[] #will quantify the fraction of contacts normalized onto the genomic fraction of each chromosome -> gives the chromosome copy number
        cov_rel_hic=[] #use the basic coverage in the corresponding HiC library to determine the enrichment per chr
        contacts = 0 
        total_contacts = len(capt_pairs[:,0])*2
        for chrom in chrs[:,0]:
            contacts = int(sum((len(capt_pairs[(capt_pairs[:,1]==str(chrom)),0]),len(capt_pairs[(capt_pairs[:,3]==str(chrom)),0]))))
            cov_chr.append(contacts)
            if total_contacts == 0:
                cov_chr_rel.append(0)
                cov_chr_norm.append(0)
                cov_rel_hic.append(0)
            else:
                cov_chr_rel.append(float(contacts/total_contacts))
                cov_chr_norm.append(float((contacts/total_contacts)/(chrs[chrs[:,0]==str(chrom),1]/sum(chrs[:,1]))))
                cov_rel_hic.append(float((contacts/total_contacts)/cov_hic[(cov_hic[:,0]==str(chrom)),3]))
        head_capt = np.array(("chr", "length", "capt_pairs", "capt_pairs_fraction_total", "capt_chr_enrichment", "capt_chr_enrich_rel_HiC"))
        cov_capt = np.column_stack((chrs[:,0], chrs[:,1], cov_chr, cov_chr_rel, cov_chr_norm, cov_rel_hic))
        cov_capt = np.row_stack((head_capt, cov_capt))
        print(cov_capt)
        np.savetxt(dir_capt + "Coverage/per_chr/" + capt_samp + "_filtered_" + str(chro) + "_" + str(pos) + "_coverage_per_chr.tsv", cov_capt, delimiter = "\t", fmt='%s')


[['chr' 'length' 'Hic_pairs' 'Hic_pairs_fraction_total'
  'Hic_chr_copy_number']
 ['chr1' 230218 423256 0.015612627882485517 0.8248833587133311]
 ['chr2' 813184 1827817 0.06742261576511858 1.0084922911882255]
 ['chr3' 316620 652192 0.0240573813576984 0.9241996896153116]
 ['chr4' 1531933 3391799 0.12511315997690875 0.9933882798175973]
 ['chr5' 576874 1239867 0.04573492660416786 0.9643236792790926]
 ['chr6' 270161 600831 0.022162830115253312 0.9978341713606509]
 ['chr7' 1090940 2389288 0.08813357506588934 0.9826442829382597]
 ['chr8' 562643 1219140 0.04497037054797426 0.9721859855036901]
 ['chr9' 439888 976229 0.03601012178230422 0.9957224191605138]
 ['chr10' 745751 1667711 0.061516791867162676 1.0033573687239568]
 ['chr11' 666816 1572389 0.058000652899222974 1.0579927228642165]
 ['chr12' 1078177 2288238 0.08440614758104527 0.9522255407309564]
 ['chr13' 924431 2095716 0.07730459593099925 1.0171537953106535]
 ['chr14' 784333 1715992 0.06329773126741756 0.9816201541257679]
 ['chr15' 109129

[['chr' 'length' 'capt_pairs' 'capt_pairs_fraction_total'
  'capt_chr_enrichment' 'capt_chr_enrich_rel_HiC']
 ['chr1' 230218 219 0.0006689147087930213 0.03534168724413963
  0.04284446627613664]
 ['chr2' 813184 2014 0.006151571796845411 0.09201382452175744
  0.09123899639663574]
 ['chr3' 316620 404 0.0012339796454446602 0.04740514307666422
  0.05129318220870223]
 ['chr4' 1531933 302165 0.9229343058559054 7.328023068461058
  7.376796381981279]
 ['chr5' 576874 1419 0.00433420078437122 0.09138688432350728
  0.09476785262789163]
 ['chr6' 270161 315 0.0009621375948392772 0.04331819378160706
  0.04341221720492714]
 ['chr7' 1090940 2846 0.008692836809246296 0.09692068416304564
  0.09863252231340285]
 ['chr8' 562643 1218 0.0037202653667118717 0.08042606293434135
  0.08272703385317014]
 ['chr9' 439888 1088 0.0033231927085242336 0.09189020528929172
  0.09228496167311737]
 ['chr10' 745751 1614 0.004929809771652678 0.08040667939050022
  0.08013762782522772]
 ['chr11' 666816 1460 0.00445943139195347

[['chr' 'length' 'capt_pairs' 'capt_pairs_fraction_total'
  'capt_chr_enrichment' 'capt_chr_enrich_rel_HiC']
 ['chr1' 230218 2823 0.008995086637054787 0.47524973585099706
  0.5761417427456662]
 ['chr2' 813184 3844 0.012248357432815656 0.18320816999661935
  0.18166541439871578]
 ['chr3' 316620 3060 0.009750253315404763 0.3745703222551341
  0.40529154733978007]
 ['chr4' 1531933 3139 0.010001975541521421 0.07941486954532548
  0.07994343315577206]
 ['chr5' 576874 3517 0.011206418598130246 0.23628801042190356
  0.24502977112264562]
 ['chr6' 270161 3422 0.010903714655331732 0.4909165039517142
  0.49198205277165286]
 ['chr7' 1090940 3849 0.012264289219278737 0.13674055178875744
  0.1391556987233283]
 ['chr8' 562643 3934 0.012535129589151092 0.2709890348811963
  0.278741968020447]
 ['chr9' 439888 4070 0.012968474180946857 0.3585936354877495
  0.3601341383777189]
 ['chr10' 745751 256280 0.816599646951612 13.318985730522812
  13.274418612644013]
 ['chr11' 666816 4401 0.014023158444802732 0.25579

[['chr' 'length' 'Hic_pairs' 'Hic_pairs_fraction_total'
  'Hic_chr_copy_number']
 ['chr1' 230218 409310 0.015370099806701214 0.8120695406142227]
 ['chr2' 813184 1786782 0.06709588740274415 1.003605161980497]
 ['chr3' 316620 627433 0.023560889868358855 0.9051262387886522]
 ['chr4' 1531933 3361926 0.12624450456315212 1.002371063504115]
 ['chr5' 576874 1239123 0.046530610497615574 0.9811007220480361]
 ['chr6' 270161 584785 0.02195940440121612 0.9886753623211841]
 ['chr7' 1090940 2344892 0.08805361236210996 0.9817527396908836]
 ['chr8' 562643 1188143 0.04461624806291906 0.9645304355732057]
 ['chr9' 439888 967473 0.03632981498201521 1.0045623139026032]
 ['chr10' 745751 1635147 0.06140180447247336 1.0014818897487034]
 ['chr11' 666816 1532341 0.05754131125039785 1.0496138495675689]
 ['chr12' 1078177 2218236 0.08329752196334728 0.9397186125209344]
 ['chr13' 924431 2083349 0.0782323472727057 1.0293609065044507]
 ['chr14' 784333 1658562 0.06228106685788762 0.9658537395261554]
 ['chr15' 1091291 2

[['chr' 'length' 'capt_pairs' 'capt_pairs_fraction_total'
  'capt_chr_enrichment' 'capt_chr_enrich_rel_HiC']
 ['chr1' 230218 431 0.0010998994523470477 0.05811249466316919
  0.07156098308922511]
 ['chr2' 813184 3983 0.010164500043383505 0.15203830081653344
  0.1514921470875097]
 ['chr3' 316620 1105 0.0028199278302633124 0.10833167528572064
  0.11968681344461188]
 ['chr4' 1531933 343637 0.8769516197359221 6.962924293283823
  6.946453810171506]
 ['chr5' 576874 2503 0.006387583130451647 0.13468257464081856
  0.1372770110286641]
 ['chr6' 270161 695 0.0017736197665457031 0.07985344835730042
  0.08076811802998989]
 ['chr7' 1090940 5358 0.01367346001316817 0.15245208537018537
  0.15528562254706485]
 ['chr8' 562643 2283 0.005826149535286101 0.12595184025916661
  0.1305835830720213]
 ['chr9' 439888 1363 0.0034783363191392713 0.09618010945048273
  0.09574329846879745]
 ['chr10' 745751 3315 0.008459783490789937 0.13798161194137804
  0.13777744096401087]
 ['chr11' 666816 2647 0.006755066938196369 0

[['chr' 'length' 'capt_pairs' 'capt_pairs_fraction_total'
  'capt_chr_enrichment' 'capt_chr_enrich_rel_HiC']
 ['chr1' 230218 2855 0.007340124126512374 0.3878108341801055
  0.4775586508105921]
 ['chr2' 813184 4705 0.01209642172162547 0.18093554987127003
  0.1802855911125596]
 ['chr3' 316620 3017 0.007756621537543899 0.2979820251786268
  0.3292159838139505]
 ['chr4' 1531933 4026 0.010350731955635312 0.08218396701161834
  0.08198956454740172]
 ['chr5' 576874 4662 0.011985869939685005 0.2527227888574857
  0.2575910741660957]
 ['chr6' 270161 3736 0.009605150170455422 0.4324514067603074
  0.4374048582995031]
 ['chr7' 1090940 4400 0.011312275361350068 0.12612608421414442
  0.12847031550312424]
 ['chr8' 562643 4322 0.011111739570853409 0.2402176667373956
  0.24905141183506346]
 ['chr9' 439888 4255 0.010939484468760124 0.3024896723608274
  0.3011158871625311]
 ['chr10' 745751 324544 0.8343934306531811 13.609207691918359
  13.589070188112183]
 ['chr11' 666816 4807 0.01235866083227495 0.225434931

In [136]:
print(capture_sites)
for site in np.arange(0, len(capture_sites),1):
    chrom = capture_sites.iloc[site,1]
    pos = capture_sites.iloc[site,2]
    pairs = pd.read_csv(str(dir_capt + "Pairs/"+ hic_samp + "_valid_idx_filtered_" + str(chrom) + "_" + str(pos) + ".pairs"), names=head, skiprows=int(len(chrs)+3), sep="\t")


                          name  chrom   start     end
0        URA-L-17213-MfeI-RC-R   chr5  101549  101551
1        URA-L-16220-MfeI-RC-F   chr5  102542  102544
2        URA-L-15683-SspI-RC-R   chr5  103079  103081
3         URA-L-9924-MfeI-RC-F   chr5  108838  108840
4         URA-L-6532-MfeI-RC-F   chr5  112230  112232
5         URA-L-6065-SspI-RC-R   chr5  112697  112699
6         URA-L-4543-SspI-RC-R   chr5  114219  114221
7         URA-L-3728-SspI-RC-R   chr5  115034  115036
8         URA-L-3081-MfeI-RC-R   chr5  115681  115683
9        URA-L-2599-MfeI-RC-R    chr5  116185  116187
10           URA-R-1439-SspI-R   chr5  120222  120224
11        URA-R-2715-93-SspI-F   chr5  121509  121511
12           URA-R-2954-SspI-F   chr5  121737  121739
13           URA-R-8073-SspI-F   chr5  126856  126858
14           URA-R-9028-MfeI-R   chr5  127811  127813
15          URA-R-10241-SspI-R   chr5  129024  129026
16          URA-R-12116-SspI-F   chr5  130899  130901
17          URA-R-13482-SspI

In [141]:
str(hic_capture_correspondance.iloc[i,1])

'AD153_S288c_DSB'

In [126]:
hic_capture_correspondance = pd.read_csv(str(dir_capt + "test.tsv"), sep = "\t")
capt_pairs = pd.read_csv(str(dir_capt + "Pairs/"+ capt_samp + "_valid_idx.pairs"), names=head, skiprows=int(len(chrs)+3), sep="\t")
print(capt_pairs)

NameError: name 'capt_samp' is not defined

Load the data. It will show an array of integer (if the matrice is not pre-normalized). Sometimes tsv or csv. check

Normalize the data in the matrice by the total number of reads. It will return an array of floats.

In [235]:
sample = input("Sample name? ")

frags = pd.read_csv(str(input_dir + "Rebin/" + sample + "_1kb.frag.tsv"), sep = "\t")
mat = pd.read_csv(str(input_dir + "Dense/" + sample + "_dense_1kb.mat.csv"), sep = " ", header = None)
mat = np.array(mat)

#print(sum(mat[:,:]))
a = 0 #number of reads
for i in range(1,len(mat)):
    a += sum(mat[i,:])
print("total number of contacts in the matrix =", a)
mat_norm=mat/a #matrix normalized
#
print("done")

Sample name? AP45
total number of contacts in the matrix = 29058650
done


Compute the cohesin peaks coverage with and without remocing cohesin at centromeres.
Defin the size of the CEN region to exclude.
Define the region (in bin) to look around the cohesin peak bin.
Also retrieve the coverage of randomly selected bins

In [57]:

print("Sample is", sample)
print("Bin size in bp is", (frags["end_pos"][0] - frags["start_pos"][0]))



#centromeres
extra_bin = input("extra kb on each side of the centromeric bin?")
region = int(input("extra bin around cohesin peak?"))
coord_cen_start = []
coord_cen_end = []
i = 0
for i in range(0, len(chrs)):
    cen = int(chrs.loc[chrs["Chr"] == "chr" + str(i+1)]["Left_arm_length"])
    cen_row=int(frags.loc[(frags["chrom"]== "chr" + str(i + 1)) & (frags["start_pos"] < cen) & (frags["end_pos"] > cen)].index.values)
    coord_cen_start.append(cen_row - int(extra_bin))
    coord_cen_end.append(cen_row + int(extra_bin) + 1)
#print(coord_cen_start)
#print(coord_cen_end)


print("==============BEFORE CEN FILTERING===================")
cohesin_row = []
i=0
for i in range(0, len(chip)):
    cohesin_row.append(int(frags.loc[(frags["chrom"]== chip["Chr"][i]) & (frags["start_pos"] < chip["Mid"][i]) & (frags["end_pos"] >= chip["Mid"][i])].index.values))
print(len(cohesin_row), "peaks before CEN filtering")
#print(coord_cohesin_start)
#print(coord_cohesin_end)

#Sum the fraction of contacts made with the rest of the genome for each bin (=row) in the cohesin peak
output_dir = "/mnt/d/OwnCloud Data/Experiments/HiC/Quantif/Cohesin_peak_coverage/"
cov_peak = 0
i=0
for i in cohesin_row:
    cov_peak += sum(mat_norm[i, ])
#print(i)
print("the peak coverage (of 1 bin) is", cov_peak)


output_dir = "/mnt/d/OwnCloud Data/Experiments/HiC/Quantif/Cohesin_peak_coverage/"
cov_region = 0
i=0
for i in cohesin_row:
    cov_region += sum(sum(mat_norm[i-region:i+region+1,: ]))
print("the region coverage of", str(region+region+1), "bin is", cov_region)





print("==============AFTER CEN FILTERING====================")
i=0
while i < len(cohesin_row):
    for j in range(0, len(coord_cen_start)):
        if (cohesin_row[i] >= coord_cen_start[j]) & (cohesin_row[i]< coord_cen_end[j]):
            cohesin_row.pop(i)
            i -= 1
    i += 1
print(len(cohesin_row), "after CEN filtering")

#Sum the fraction of contacts made with the rest of the genome for each bin (=row) in the cohesin peak
output_dir = "/mnt/d/OwnCloud Data/Experiments/HiC/Quantif/Cohesin_peak_coverage/"
cov_peak = 0
i=0
for i in cohesin_row:
    cov_peak += sum(mat_norm[i, ])
print(i)
print("the peak coverage (of 1 bin) is", cov_peak)


output_dir = "/mnt/d/OwnCloud Data/Experiments/HiC/Quantif/Cohesin_peak_coverage/"
cov_region = 0
i=0
for i in cohesin_row:
    cov_region += sum(sum(mat_norm[i-region:i+region+1,: ]))
print("the region coverage of", str(region+region+1), "bin is", cov_region)




print("=====================RANDOM REGIONS===================")
#select random peak regions and perform the same calculation
rdm_row = np.random.randint(region, len(mat_norm)-region, len(cohesin_row))

cov_rdm = 0
i=0
for i in rdm_row:
    cov_rdm += sum(mat_norm[i, ])
#print(i)
print("the rdm coverage (of 1 bin) is", cov_rdm)


output_dir = "/mnt/d/OwnCloud Data/Experiments/HiC/Quantif/Cohesin_peak_coverage/"
cov_rdm_region = 0
i=0
for i in rdm_row:
    cov_rdm_region += sum(sum(mat_norm[i-region:i+region+1,: ]))
print("the random region coverage of", str(region+region+1), "bin is", cov_rdm_region)



print("done")
#np.savetxt(output_dir + str(sample) + "_cohesin_peak_coverage_1kb.tsv", cov_region, delimiter = "\t")


NameError: name 'sample' is not defined

Input a number of bins to consider around the center of the cohesin peak.

Define the lines in frags corresponding to each cohesin peak in chrs.

aggregate the absolute fraction of contacts centered on cohesin peaks in the window defined (in bins)

In [216]:
#define cohesin peaks regions
cohesin_window = input ("bins on each side to look around center of cohesin peak?")
coord_cohesin_start = []
coord_cohesin_end = []
i=0
for i in range(0, len(chip)):
    cohesin_row = int(frags.loc[(frags["chrom"]== chip["Chr"][i]) & (frags["start_pos"] < chip["Mid"][i]) & (frags["end_pos"] >= chip["Mid"][i])].index.values)
    coord_cohesin_start.append(cohesin_row - int(cohesin_window))
    coord_cohesin_end.append(cohesin_row + int(cohesin_window) + 1)
#print(coord_cohesin_start)
#print(coord_cohesin_end)

#Sum the fraction of contacts made with the rest of the genome for each bin (=row) in the cohesin peak
output_dir = "/mnt/d/OwnCloud Data/Experiments/HiC/Quantif/Cohesin_peak_coverage/"
cov_region = [0]*(coord_cohesin_end[0]-coord_cohesin_start[0])
i=0
for i in coord_cohesin_start:
    j=0
    while j < len(cov_region):
        coord = i + j
        if coord < len(mat_norm):
            cov_region[j] += sum(mat_norm[coord, ])
            j +=1
        else:
            j=len(cov_region)

print(i)
#print(cov_region)
print("done")
np.savetxt(output_dir + sample + "_cohesin_coverage_bins_" + str(cohesin_window) + ".tsv", cov_region, delimiter = "\t")


bins on each side to look around center of cohesin peak?20
8308
done
